# k-Nearest-Neighbour classifiers
Here we use the k-nearest-neighbor method on the converted data from the descriptive_analysis.ipynb.
As reference we use dataMining_R_8_ml.Rmd (./uebung/knn_uebung/) from the lecture.

In [145]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import utils
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor  
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

In [146]:
test = pd.read_csv('./data/converted_test_2020_07_01_20-34-00.csv', index_col=0)
train = pd.read_csv('./data/converted_train_2020_07_01_20-34-00.csv', index_col=0)
train

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,min_future_price,price_in_eur,min_future_price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday,flight_distance_km,Request_hour
0,06/03/19,06/05/19,56.0,19,2019-06-05 FR 146,SXF-STN,FR 146,208.07,€,259.07,208.070000,259.070000,Monday,Morning,Evening,907.2,11
1,06/03/19,06/05/19,44.0,19,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,259.070000,259.070000,Monday,Night,Evening,907.2,23
2,06/04/19,06/05/19,32.0,19,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,259.070000,259.070000,Tuesday,Morning,Evening,907.2,11
3,06/04/19,06/05/19,20.0,19,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,259.070000,259.070000,Tuesday,Night,Evening,907.2,23
4,06/03/19,06/05/19,58.0,21,2019-06-05 FR 147,STN-SXF,FR 147,127.49,£,223.37,112.979088,197.946027,Monday,Morning,Night,907.2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83619,08/01/19,09/10/19,959.0,10,2019-09-10 FR 8543,SXF-STN,FR 8543,35.69,€,39.69,35.690000,39.690000,Thursday,Morning,Morning,907.2,11
83620,08/01/19,09/10/19,947.0,10,2019-09-10 FR 8543,SXF-STN,FR 8543,46.83,€,39.69,46.830000,39.690000,Thursday,Night,Morning,907.2,23
83621,08/02/19,09/10/19,935.0,10,2019-09-10 FR 8543,SXF-STN,FR 8543,46.83,€,39.69,46.830000,39.690000,Friday,Morning,Morning,907.2,11
83622,08/02/19,09/10/19,923.0,10,2019-09-10 FR 8543,SXF-STN,FR 8543,39.69,€,39.69,39.690000,39.690000,Friday,Night,Morning,907.2,23


In [147]:
# Check for nulls
print(train.isnull().values.any())
print(data.isnull().values.any())

False
False


In [148]:
# Scale numeric columns
# scaler = preprocessing.MinMaxScaler()

# Compute logarithmic values
trainFeatures = [['min_future_price_in_eur', 'Duration_Time', 'price_in_eur', 'flight_distance_km']]
for feature in trainFeatures:
    train[feature] = np.log1p(train[feature])
    
testFeatures = [['Duration_Time', 'price_in_eur', 'flight_distance_km']]
for feature in testFeatures:
    test[feature] = np.log1p(test[feature])

In [149]:
# Hot encoding for categorical values
train = pd.get_dummies(train, columns=['Request_Date', 'Flight_Date', 'flight_number', 'Request_Weekday', 'Request_timeofday', 'Departure_timeofday'], drop_first=True)

In [150]:
# Drop columns we do not need from train
train_dropped = train.drop(columns=['route_abb', 'Currency', 'flight_unique_id', 'Price', 'min_future_price'])

In [151]:
# Generate test and target / Figure out which features to drop. 'min_future_price_in_eur' needs to be dropped!
x = train_dropped.drop(columns=['min_future_price_in_eur'])
y = train_dropped['min_future_price_in_eur'].values

# Build model

In [152]:
# Writing a function to evaluate model performance using KFOLD and cross validation
def rmsle_cv(model):
    kfold=KFold(n_splits=2, shuffle=True, random_state=10).get_n_splits(x)
    rmse=np.sqrt(-cross_val_score(model,x,y,cv=kfold, scoring='neg_mean_squared_error'))
    return rmse

In [153]:
lightgbm=LGBMRegressor(objective='regression',num_leaves=80,
                              learning_rate=0.1, n_estimators=1200,
                              max_bin = 30, bagging_fraction = 0.8,
                              bagging_freq = 9, feature_fraction = 0.129,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =3, min_sum_hessian_in_leaf = 6, random_state=10)

In [154]:
xgb=XGBRegressor(learning_rate =0.1, 
      n_estimators=1500, max_depth=12, min_child_weight=1,gamma=0, reg_alpha=2e-5,
      subsample=0.8,colsample_bytree=0.8,
      nthread=4,scale_pos_weight=1,seed=27,verbose=True,random_state=10)

In [155]:
grb=GradientBoostingRegressor(learning_rate=0.1,n_estimators=400, max_depth=12
                              ,subsample=0.8,
                              verbose=False,random_state=10)

In [156]:
svr = Pipeline([('Scaler',RobustScaler()), ('SVR',SVR(C= 10000, epsilon= 0.008, gamma=0.009))])

In [157]:
krr=KernelRidge(alpha=1, kernel='polynomial', gamma=0.001,degree=3,coef0=5)

In [158]:
rf=RandomForestRegressor(n_estimators=3000, oob_score = False, n_jobs = -1,random_state =50,
                         max_features = "auto", min_samples_leaf = 2,warm_start=True,criterion='mse',max_depth=50)

In [159]:
# Final ensemble that uses outputs from each model regressor and the original data in the meta regressor XGB
avg=StackingCVRegressor(regressors=(lightgbm,grb,svr,krr,rf),meta_regressor=xgb, use_features_in_secondary=True)

In [160]:
# Function to calculate RMSE error of the complete model for reference, similiar to rmsle_cv without KFOLD CV
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
X=np.array(x)
Y=np.array(y)
# Fitting the model on the train data and predicting the min future price values
avg.fit(X,Y)
y_pred=avg.predict(X)

In [ ]:
# Plot of the model prediction with the original price values in the training data set to undestand model fit
plt.scatter(list(y_pred),y)
plt.plot(range(7,13),range(7,13),'r')

In [ ]:
# RMSE value of the model on training data
rmsle(y,y_pred)

In [ ]:
# Predictions of test data
Predict=avg.predict(np.array(test_data.drop('min_future_price_in_eur',axis=1)))
# Converting price back to original scale and making it integer
Predict=np.exp(Predict)
Predict=Predict.astype(int)
# Converting predicted values to dataframe
Submission=pd.DataFrame(Predict,columns=['Price'])
# Final Submission
Submission.to_excel('Submission_rf.xlsx')